<a href="https://colab.research.google.com/github/tarandeepkhurana/Conversation_Management_and_Classification_System/blob/main/ConversationManagement%26Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Conversation Management and Classification System**

### **Task 1: Managing Conversation History with Summarization**

In [1]:
# Initialize the Groq client using the OpenAI-compatible API.
# The API key is securely retrieved from Google Colab's `userdata` storage
# to avoid hardcoding sensitive information into the notebook or pushing it to GitHub.

import openai
from google.colab import userdata

groq_api_key = userdata.get("GROQ_API_KEY") # Replace this with your own key for testing. For security, I don’t commit my real key.

client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=groq_api_key
)

### **ConversationManager:** A lightweight class to manage user–assistant chat history.
### **Supports:**
- Adding messages with role and timestamp
- Truncation by number of turns or total character length
- Periodic summarization of history using a model (with fallback)

In [17]:
import time

class ConversationManager:
  """
  Manages messages and performs periodic summarization.

  Each message is stored as a dict: {'role': 'user'|'assistant'|'system', 'content': str, 'timestamp': float}
  """

  def __init__(self, summarization_k: int = 3, summarization_model: str = "llama-3.3-70b-versatile"):
    self.history = []                               # Keeps the history of conversations
    self.run_count = 0                              # Keeps the count of messages exchanged
    self.total_chars = 0                            # Keeps the count of total characters in the history
    self.summarization_k = summarization_k          # Sets the threshold for periodic summarization
    self.summarization_model = summarization_model  # Sets the model used for summarization

  def add_message(self, role: str, content: str) -> None:
    """
    Add a single message to the conversation history.

    The message is stored as a dictionary with the role, content,
    and the timestamp of when it was added.

    Parameters
    ----------
    role : str
        The role of the message sender (e.g., 'user' or 'assistant').
    content : str
        The text content of the message.

    Returns
    -------
    None
    """
    self.total_chars += len(content)   # Maintaining the total characters present in the history for truncation by character length

    self.history.append({
        'role': role,
        'content': content,
        'timestamp': time.time()
    })

  def get_history(self) -> list[dict[str, str]]:
    """Returns the 'history' list of conversations"""
    return self.history

  def truncate_by_turns(self, n_turns: int, update: bool = False) -> list[dict[str, str]]:
    """
    Truncate the conversation history to the last `n_turns` entries.

    Parameters
    ----------
    n_turns : int
        The number of most recent conversation turns to keep.
    update : bool, optional
        If True, update `self.history` with the truncated history.
        If False (default), `self.history` remains unchanged.

    Returns
    -------
    list
        A list containing the last `n_turns` entries of the conversation history.
    """
    if len(self.history) > n_turns:
      truncated = self.history[-n_turns:]
      if update:
        self.history = truncated
      return truncated
    return []

  def truncate_by_chars(self, max_chars: int, update: bool = False) -> list[dict[str, str]]:
    """
    Truncate the conversation history by a maximum total character length.

    This method checks whether the total characters in history exceeds the maximum
    character length mentioned and iterates backward through the conversation history,
    collecting complete messages until the cumulative length of their content reaches
    the specified character budget (`max_chars`).

    - Messages are never partially truncated: if adding a message would cause
      the total character count to exceed the budget, that message and all
      earlier ones are excluded.
    - The returned list preserves chronological order (oldest to newest).

    Parameters
    ----------
    max_chars : int
        The maximum allowed total number of characters across all included
        message contents.
    update : bool, optional
        If True, update `self.history` with the truncated history.
        If False (default), `self.history` remains unchanged.

    Returns
    -------
    List[Dict[str, Any]]
        A list of message dictionaries (with keys: 'role', 'content',
        'timestamp') that fit within the character limit.
    """
    if self.total_chars > max_chars:
      acc = []
      total = 0
      for msg in reversed(self.history):
          content = msg['content']
          if total + len(content) > max_chars:
              break
          acc.append(msg)
          total += len(content)
      truncated = list(reversed(acc))
      if update:
          self.history = truncated
      return truncated
    return []

  def summarize_history(self, prompt_extra: str = None) -> str:
    """
    Call the model to summarize self.history into a concise summary.

    Parameters
    ----------
    prompt_extra : Optional[str]
        Extra text to prepend to the summarization prompt.

    Returns
    -------
    str
        The summary string.
    """
    # Build a single string from history (formatting is flexible)
    text = "\n---\n".join([f"{m['role']}: {m['content']}" for m in self.history])
    if prompt_extra:
        text = prompt_extra + "\n\n" + text

    system_msg = {
        'role': 'system',
        'content': 'You are a helpful summarizer. Produce a concise bullet-point summary.'
    }
    user_msg = {
        'role': 'user',
        'content': f'Summarize the conversation below concisely:\n\n{text}'
    }

    try:
        resp = client.chat.completions.create(
            model=self.summarization_model,
            messages=[system_msg, user_msg],
            max_completion_tokens=1024,
            temperature=0.2,
        )
        summary = resp.choices[0].message.content
    except Exception as e:
        print('Summarization call failed:', e)
        # Fallback: naive extractive summary (last few turns)
        last = self.history[-6:]
        summary = '\n'.join([
            f"- {m['role']}: {m['content'][:120]}{'...' if len(m['content']) > 120 else ''}"
            for m in last
        ])

    return summary

  def maybe_summarize(self):
    """
    Periodically summarize the conversation history based on the configured interval.

    This method increments the internal run counter each time it is called. If the
    counter reaches a multiple of `self.summarization_k`, a summarization of the
    current conversation history is performed. The existing history is then replaced
    with a single system message containing the summary, ensuring that the
    conversation context remains concise.

    Returns
    -------
    str or None
        The generated summary string if summarization was performed, otherwise None.
    """

    self.run_count += 1
    if self.summarization_k > 0 and self.run_count % self.summarization_k == 0:
      print(f"Performing periodic summarization at run {self.run_count}...")
      summary = self.summarize_history()
      # Replace the entire history with a single 'system' summary message + keep recent context
      self.history = [
      {'role': 'system', 'content': f'[SUMMARY]\n{summary}', 'timestamp': time.time()}
      ]
      return summary
    return None


## **Task 1 Demonstration**

### **# Feeding multiple conversation samples and showing how summarization happens after every 3rd run**

In [13]:
cm = ConversationManager(summarization_k=3) # Setting periodic summarization to take place after every 3rd run.

samples = [
  ('user', 'Hi, I need help booking a flight to Delhi next week.'),
  ('assistant', 'Sure — when are you planning to travel and from which city?'),
  ('user', 'I want to fly on Oct 10 from Mumbai. Prefer morning flights.'),
  ('assistant', 'Got it. Do you have a budget or airline preference?'),
  ('user', 'Budget around 10k INR, no strong preference.'),
  ('assistant', 'I found flights with SpiceJet and Indigo around 9.5k. Want me to book?'),
  ('user', 'Yes, please book the SpiceJet one.'),
  ('assistant', 'Done. Your ticket has been reserved. Do you need hotel options too?'),
  ('user', 'Yes, I’ll need a hotel in Delhi near Connaught Place.'),
  ('assistant', 'Okay, do you prefer 3-star or 5-star accommodations?'),
  ('user', '3-star, with free Wi-Fi and breakfast included.'),
  ('assistant', 'I found Hotel Bright and Hotel Palace Heights around 3.5k per night.'),
  ('user', 'Hotel Bright looks fine, please book it.'),
  ('assistant', 'Hotel Bright is confirmed. Need airport pickup as well?'),
  ('user', 'Yes, please arrange a cab for me.'),
  ('assistant', 'Cab booked for Oct 10, pickup at 6 AM. Anything else you’d like me to plan?'),
  ('user', 'Yes, also suggest a few tourist spots I can visit in Delhi.'),
  ('assistant', 'You could visit India Gate, Qutub Minar, and Lotus Temple. Want me to create an itinerary?'),
  ('user', 'Yes, that would be great.'),
  ('assistant', 'Alright, I’ll prepare a 2-day itinerary covering the highlights. Would you like me to also recommend restaurants?'),
  ('user', 'Yes, especially places with good North Indian food.'),
  ('assistant', 'Sure — I’ll add Karim’s and Gulati to your itinerary.'),
]


print('Feeding messages and showing summaries...')
for i, (r, c) in enumerate(samples, 1):
  cm.add_message(r, c)               # Calling the add_message function
  print(f"Added: {r}: {c[:60]}")
  summary = cm.maybe_summarize()     # Checking for the periodic summarization
  if summary:
    print('\n--- SUMMARY GENERATED ---')
    print(summary)
    print('--- END SUMMARY ---\n')

Feeding messages and showing summaries...
Added: user: Hi, I need help booking a flight to Delhi next week.
Added: assistant: Sure — when are you planning to travel and from which city?
Added: user: I want to fly on Oct 10 from Mumbai. Prefer morning flights.
Performing periodic summarization at run 3...

--- SUMMARY GENERATED ---
Here's a concise summary:
* User needs help booking a flight to Delhi
* Travel date: October 10
* Departure city: Mumbai
* Preferred flight time: Morning
--- END SUMMARY ---

Added: assistant: Got it. Do you have a budget or airline preference?
Added: user: Budget around 10k INR, no strong preference.
Added: assistant: I found flights with SpiceJet and Indigo around 9.5k. Want m
Performing periodic summarization at run 6...

--- SUMMARY GENERATED ---
Here's a concise summary:
* User wants to book a Mumbai to Delhi flight on October 10
* Preferred morning flight with a budget of 10k INR
* Options found with SpiceJet and Indigo for around 9.5k INR
--- END SUMMA

In [15]:
print('Current history:')
for m in cm.get_history():
  print(m['role'], ':', m['content'])

Current history:
system : [SUMMARY]
Here's a concise summary:
* Booked Mumbai to Delhi SpiceJet flight and Hotel Bright
* Arranged airport pickup on Oct 10
* Identified tourist spots: India Gate, Qutub Minar, and Lotus Temple
* Creating a 2-day itinerary with North Indian food recommendations
assistant : Sure — I’ll add Karim’s and Gulati to your itinerary.


### **# Feeding multiple conversation samples and showing how truncation works with different numbers of conversation turns.**

In [22]:
# Setting the number of conversation turns to keep in history as 3.
cm2 = ConversationManager()

samples = [
  ('user', 'Hi, I need help booking a flight to Delhi next week.'),
  ('assistant', 'Sure — when are you planning to travel and from which city?'),
  ('user', 'I want to fly on Oct 10 from Mumbai. Prefer morning flights.'),
  ('assistant', 'Got it. Do you have a budget or airline preference?'),
  ('user', 'Budget around 10k INR, no strong preference.'),
  ('assistant', 'I found flights with SpiceJet and Indigo around 9.5k. Want me to book?'),
  ('user', 'Yes, please book the SpiceJet one.'),
  ('assistant', 'Done. Your ticket has been reserved. Do you need hotel options too?'),
  ('user', 'Yes, I’ll need a hotel in Delhi near Connaught Place.'),
  ('assistant', 'Okay, do you prefer 3-star or 5-star accommodations?'),
  ('user', '3-star, with free Wi-Fi and breakfast included.'),
  ('assistant', 'I found Hotel Bright and Hotel Palace Heights around 3.5k per night.'),
  ('user', 'Hotel Bright looks fine, please book it.'),
  ('assistant', 'Hotel Bright is confirmed. Need airport pickup as well?'),
  ('user', 'Yes, please arrange a cab for me.'),
  ('assistant', 'Cab booked for Oct 10, pickup at 6 AM. Anything else you’d like me to plan?'),
  ('user', 'Yes, also suggest a few tourist spots I can visit in Delhi.'),
  ('assistant', 'You could visit India Gate, Qutub Minar, and Lotus Temple. Want me to create an itinerary?'),
  ('user', 'Yes, that would be great.'),
  ('assistant', 'Alright, I’ll prepare a 2-day itinerary covering the highlights. Would you like me to also recommend restaurants?'),
  ('user', 'Yes, especially places with good North Indian food.'),
  ('assistant', 'Sure — I’ll add Karim’s and Gulati to your itinerary.'),
]


print('Feeding messages and showing history after truncation...\n')
for i, (r, c) in enumerate(samples, 1):
  cm2.add_message(r, c)               # Calling the add_message function
  print(f"Added: {r}: {c[:60]}")
  if cm2.truncate_by_turns(n_turns=3, update=True):       # Displays the latest history and updates it by number of conversations
    print('\nTruncate by last 3 turns:')
    for m in cm2.history:
      print('-', m['role'], ':', m['content'])
    print("\n")

Feeding messages and showing history after truncation...

Added: user: Hi, I need help booking a flight to Delhi next week.
Added: assistant: Sure — when are you planning to travel and from which city?
Added: user: I want to fly on Oct 10 from Mumbai. Prefer morning flights.
Added: assistant: Got it. Do you have a budget or airline preference?

Truncate by last 3 turns:
- assistant : Sure — when are you planning to travel and from which city?
- user : I want to fly on Oct 10 from Mumbai. Prefer morning flights.
- assistant : Got it. Do you have a budget or airline preference?


Added: user: Budget around 10k INR, no strong preference.

Truncate by last 3 turns:
- user : I want to fly on Oct 10 from Mumbai. Prefer morning flights.
- assistant : Got it. Do you have a budget or airline preference?
- user : Budget around 10k INR, no strong preference.


Added: assistant: I found flights with SpiceJet and Indigo around 9.5k. Want m

Truncate by last 3 turns:
- assistant : Got it. Do you hav

In [24]:
# Setting the number of conversation turns to keep in history as 4
cm2 = ConversationManager()

samples = [
  ('user', 'Hi, I need help booking a flight to Delhi next week.'),
  ('assistant', 'Sure — when are you planning to travel and from which city?'),
  ('user', 'I want to fly on Oct 10 from Mumbai. Prefer morning flights.'),
  ('assistant', 'Got it. Do you have a budget or airline preference?'),
  ('user', 'Budget around 10k INR, no strong preference.'),
  ('assistant', 'I found flights with SpiceJet and Indigo around 9.5k. Want me to book?'),
  ('user', 'Yes, please book the SpiceJet one.'),
  ('assistant', 'Done. Your ticket has been reserved. Do you need hotel options too?'),
  ('user', 'Yes, I’ll need a hotel in Delhi near Connaught Place.'),
  ('assistant', 'Okay, do you prefer 3-star or 5-star accommodations?'),
  ('user', '3-star, with free Wi-Fi and breakfast included.'),
  ('assistant', 'I found Hotel Bright and Hotel Palace Heights around 3.5k per night.'),
  ('user', 'Hotel Bright looks fine, please book it.'),
  ('assistant', 'Hotel Bright is confirmed. Need airport pickup as well?'),
  ('user', 'Yes, please arrange a cab for me.'),
  ('assistant', 'Cab booked for Oct 10, pickup at 6 AM. Anything else you’d like me to plan?'),
  ('user', 'Yes, also suggest a few tourist spots I can visit in Delhi.'),
  ('assistant', 'You could visit India Gate, Qutub Minar, and Lotus Temple. Want me to create an itinerary?'),
  ('user', 'Yes, that would be great.'),
  ('assistant', 'Alright, I’ll prepare a 2-day itinerary covering the highlights. Would you like me to also recommend restaurants?'),
  ('user', 'Yes, especially places with good North Indian food.'),
  ('assistant', 'Sure — I’ll add Karim’s and Gulati to your itinerary.'),
]


print('Feeding messages and showing history after truncation...\n')
for i, (r, c) in enumerate(samples, 1):
  cm2.add_message(r, c)               # Calling the add_message function
  print(f"Added: {r}: {c[:60]}")
  if cm2.truncate_by_turns(n_turns=4, update=True):       # Displays the latest history and updates it by number of conversations
    print('\nTruncate by last 4 turns:')
    for m in cm2.history:
      print('-', m['role'], ':', m['content'])
    print("\n")

Feeding messages and showing history after truncation...

Added: user: Hi, I need help booking a flight to Delhi next week.
Added: assistant: Sure — when are you planning to travel and from which city?
Added: user: I want to fly on Oct 10 from Mumbai. Prefer morning flights.
Added: assistant: Got it. Do you have a budget or airline preference?
Added: user: Budget around 10k INR, no strong preference.

Truncate by last 4 turns:
- assistant : Sure — when are you planning to travel and from which city?
- user : I want to fly on Oct 10 from Mumbai. Prefer morning flights.
- assistant : Got it. Do you have a budget or airline preference?
- user : Budget around 10k INR, no strong preference.


Added: assistant: I found flights with SpiceJet and Indigo around 9.5k. Want m

Truncate by last 4 turns:
- user : I want to fly on Oct 10 from Mumbai. Prefer morning flights.
- assistant : Got it. Do you have a budget or airline preference?
- user : Budget around 10k INR, no strong preference.
- assis

### **# Feeding multiple conversation samples to demonstrate truncation by character length at different limits.**

In [25]:
# Setting the max character length as 200
cm3 = ConversationManager()

samples = [
  ('user', 'Hi, I need help booking a flight to Delhi next week.'),
  ('assistant', 'Sure — when are you planning to travel and from which city?'),
  ('user', 'I want to fly on Oct 10 from Mumbai. Prefer morning flights.'),
  ('assistant', 'Got it. Do you have a budget or airline preference?'),
  ('user', 'Budget around 10k INR, no strong preference.'),
  ('assistant', 'I found flights with SpiceJet and Indigo around 9.5k. Want me to book?'),
  ('user', 'Yes, please book the SpiceJet one.'),
  ('assistant', 'Done. Your ticket has been reserved. Do you need hotel options too?'),
  ('user', 'Yes, I’ll need a hotel in Delhi near Connaught Place.'),
  ('assistant', 'Okay, do you prefer 3-star or 5-star accommodations?'),
  ('user', '3-star, with free Wi-Fi and breakfast included.'),
  ('assistant', 'I found Hotel Bright and Hotel Palace Heights around 3.5k per night.'),
  ('user', 'Hotel Bright looks fine, please book it.'),
  ('assistant', 'Hotel Bright is confirmed. Need airport pickup as well?'),
  ('user', 'Yes, please arrange a cab for me.'),
  ('assistant', 'Cab booked for Oct 10, pickup at 6 AM. Anything else you’d like me to plan?'),
  ('user', 'Yes, also suggest a few tourist spots I can visit in Delhi.'),
  ('assistant', 'You could visit India Gate, Qutub Minar, and Lotus Temple. Want me to create an itinerary?'),
  ('user', 'Yes, that would be great.'),
  ('assistant', 'Alright, I’ll prepare a 2-day itinerary covering the highlights. Would you like me to also recommend restaurants?'),
  ('user', 'Yes, especially places with good North Indian food.'),
  ('assistant', 'Sure — I’ll add Karim’s and Gulati to your itinerary.'),
]


print('Feeding messages and showing history after truncation...\n')
for i, (r, c) in enumerate(samples, 1):
  cm3.add_message(r, c)               # Calling the add_message function
  print(f"Added: {r}: {c[:60]}")
  if cm3.truncate_by_chars(max_chars=200, update=True):     # Displays the latest history and updates it by max character length
    print('\nTruncate by last 200 characters:')
    for m in cm3.history:
      print('-', m['role'], ':', m['content'])
    print("\n")

Feeding messages and showing history after truncation...

Added: user: Hi, I need help booking a flight to Delhi next week.
Added: assistant: Sure — when are you planning to travel and from which city?
Added: user: I want to fly on Oct 10 from Mumbai. Prefer morning flights.
Added: assistant: Got it. Do you have a budget or airline preference?

Truncate by last 200 characters:
- assistant : Sure — when are you planning to travel and from which city?
- user : I want to fly on Oct 10 from Mumbai. Prefer morning flights.
- assistant : Got it. Do you have a budget or airline preference?


Added: user: Budget around 10k INR, no strong preference.

Truncate by last 200 characters:
- user : I want to fly on Oct 10 from Mumbai. Prefer morning flights.
- assistant : Got it. Do you have a budget or airline preference?
- user : Budget around 10k INR, no strong preference.


Added: assistant: I found flights with SpiceJet and Indigo around 9.5k. Want m

Truncate by last 200 characters:
- assistant

In [26]:
# Setting the max character length as 300
cm3 = ConversationManager()

samples = [
  ('user', 'Hi, I need help booking a flight to Delhi next week.'),
  ('assistant', 'Sure — when are you planning to travel and from which city?'),
  ('user', 'I want to fly on Oct 10 from Mumbai. Prefer morning flights.'),
  ('assistant', 'Got it. Do you have a budget or airline preference?'),
  ('user', 'Budget around 10k INR, no strong preference.'),
  ('assistant', 'I found flights with SpiceJet and Indigo around 9.5k. Want me to book?'),
  ('user', 'Yes, please book the SpiceJet one.'),
  ('assistant', 'Done. Your ticket has been reserved. Do you need hotel options too?'),
  ('user', 'Yes, I’ll need a hotel in Delhi near Connaught Place.'),
  ('assistant', 'Okay, do you prefer 3-star or 5-star accommodations?'),
  ('user', '3-star, with free Wi-Fi and breakfast included.'),
  ('assistant', 'I found Hotel Bright and Hotel Palace Heights around 3.5k per night.'),
  ('user', 'Hotel Bright looks fine, please book it.'),
  ('assistant', 'Hotel Bright is confirmed. Need airport pickup as well?'),
  ('user', 'Yes, please arrange a cab for me.'),
  ('assistant', 'Cab booked for Oct 10, pickup at 6 AM. Anything else you’d like me to plan?'),
  ('user', 'Yes, also suggest a few tourist spots I can visit in Delhi.'),
  ('assistant', 'You could visit India Gate, Qutub Minar, and Lotus Temple. Want me to create an itinerary?'),
  ('user', 'Yes, that would be great.'),
  ('assistant', 'Alright, I’ll prepare a 2-day itinerary covering the highlights. Would you like me to also recommend restaurants?'),
  ('user', 'Yes, especially places with good North Indian food.'),
  ('assistant', 'Sure — I’ll add Karim’s and Gulati to your itinerary.'),
]


print('Feeding messages and showing history after truncation...\n')
for i, (r, c) in enumerate(samples, 1):
  cm3.add_message(r, c)               # Calling the add_message function
  print(f"Added: {r}: {c[:60]}")
  if cm3.truncate_by_chars(max_chars=300, update=True):     # Displays the latest history and updates it by max character length
    print('\nTruncate by last 300 characters:')
    for m in cm3.history:
      print('-', m['role'], ':', m['content'])
    print("\n")

Feeding messages and showing history after truncation...

Added: user: Hi, I need help booking a flight to Delhi next week.
Added: assistant: Sure — when are you planning to travel and from which city?
Added: user: I want to fly on Oct 10 from Mumbai. Prefer morning flights.
Added: assistant: Got it. Do you have a budget or airline preference?
Added: user: Budget around 10k INR, no strong preference.
Added: assistant: I found flights with SpiceJet and Indigo around 9.5k. Want m

Truncate by last 300 characters:
- assistant : Sure — when are you planning to travel and from which city?
- user : I want to fly on Oct 10 from Mumbai. Prefer morning flights.
- assistant : Got it. Do you have a budget or airline preference?
- user : Budget around 10k INR, no strong preference.
- assistant : I found flights with SpiceJet and Indigo around 9.5k. Want me to book?


Added: user: Yes, please book the SpiceJet one.

Truncate by last 300 characters:
- user : I want to fly on Oct 10 from Mumbai. Pref



---





### **Task 2: JSON Schema Classification & Information Extraction**

In [9]:
from jsonschema import validate, ValidationError
import json
import re

MODEL = "llama-3.3-70b-versatile"

JSON_SCHEMA = {                # This is the required output schema
  "type": "object",
  "properties": {
    "name": {"type": ["string", "null"]},
    "email": {"type": ["string", "null"], "format": "email"},
    "phone": {"type": ["string", "null"]},
    "location": {"type": ["string", "null"]},
    "age": {"type": ["integer", "null"], "minimum": 0, "maximum": 120}
  },
  "required": ["name", "email", "phone", "location", "age"]   # force all keys
}

def validate_extraction(parsed):              # Helper function to validate whether the output is in the valid JSON schema
    try:
        validate(instance=parsed, schema=JSON_SCHEMA)
        return True, None
    except ValidationError as e:
        return False, str(e)


def extract_info(args):                      # Tool function: The function that the LLM calls upon requirement
  valid, err = validate_extraction(args)     # Validate schema
  if not valid:
      return {"error": err}
  return args

def run_conversation(user_prompt: str):
    messages = [
        {"role": "system", "content": "You are an assistant that extracts contact info from text using the available tool."},
        {"role": "user", "content": user_prompt},
    ]

    tools = [
        {
          "type": "function",
          "function": {
              "name": "extract_info",
              "description": "Extract structured personal information from user messages.",
              "parameters": JSON_SCHEMA
           },
        }
    ]

    # 1. First call: LLM reads user message and decides to call our tool
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_completion_tokens=1024,
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    # 2. If tool was called -> run the function in Python
    if tool_calls:
        available_functions = {"extract_info": extract_info}
        messages.append(response_message)  # log the model’s function call request

        for tool_call in tool_calls:
            fn_name = tool_call.function.name
            fn_to_call = available_functions[fn_name]
            fn_args = json.loads(tool_call.function.arguments)

            # Execute our Python tool -> actually parses the info
            function_response = fn_to_call(fn_args)
            if "error" in function_response:
              return function_response

            # Add the tool output to the conversation
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": fn_name,
                "content": json.dumps(function_response),
            })

        # 3. Second call: LLM sees the parsed JSON and responds naturally
        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages + [
                {"role": "system", "content": "Respond strictly in JSON, no explanations or text."}
            ],
        )
        return second_response.choices[0].message.content

### **Task 2 Demonstration**

In [10]:
user_prompt = """
  Hey, I’m Arjun Kapoor. You can reach me at arjun.k@example.com
  or call me on +91-9876543210.
  I’m currently living in Bengaluru, but I was born in Delhi.
  I’m 29 years old and work as a software engineer at Infosys.
  Sometimes I travel for work, so email is usually the best way to contact me.
"""
print(run_conversation(user_prompt))

{"age": 29, "email": "arjun.k@example.com", "location": "Bengaluru", "name": "Arjun Kapoor", "phone": "+91-9876543210"}


In [11]:
user_prompt = """
  Hi, I’m Tarandeep Singh Khurana, a 20-year-old student currently living in Indore, Madhya Pradhesh.
  You can email me at tarandeepkhurana2005@gmail.com
  or call me on +91-8989244098.
  I usually check my email daily, but phone calls are faster if urgent.
"""
print(run_conversation(user_prompt))

{"age":20,"email":"tarandeepkhurana2005@gmail.com","location":"Indore, Madhya Pradesh","name":"Tarandeep Singh Khurana","phone":"+91-8989244098"}


In [12]:
user_prompt = """
  Hi, this is Priya Mehta. I’m based in Pune and I’m 27 years old.
  If you need to reach me, drop an email at priya.mehta27@yahoo.com
  I don’t usually share my phone number here.
"""
print(run_conversation(user_prompt))

{"age":27,"email":"priya.mehta27@yahoo.com","location":"Pune","name":"Priya Mehta","phone":null}
